<div class = "alert  alert-danger"> 
    
Attention: **veillez à bien sauvegarder votre travail** dans le bon dossier du disque réseau (dossier document) avec le bon nom (et l'extension *.ipynb*), **sinon toutes les modifications seront perdues!**

Pour reprendre votre travail, il suffit d'ouvrir le fichier .ipynb en cliquant sur le bouton "Ouvrir"  &nbsp; <i class="fa fa-folder" aria-hidden="true"> 
</div>

# Algorithme de chiffrement RSA

## Étape 1 : générer une clé publique et privée

### Fonction pour l'étape 1.1)

In [1]:
from math import sqrt

def primalite(n):
    ''' Test si un nombre est premier en évitant les diviseurs pairs'''
    if n == 1 or n%2 == 0:
        return False
    else:
        for d in range(3,int(sqrt(n))+1,2):
            if n%d == 0:
                return False
        return  True


In [2]:
def premier_sup(n):
    ''' Donne le prochain nombre premier >= à n'''
    p = n
    while not primalite(p):
        p = p + 1
    
    return p

### Fonction pour l'étape 1.3)

In [3]:
def euclide_etendu(a,b):
    """Calcul PGCD et les coefficients de Bezout avec l'algorithme
       d'Euclide étendu 
    """
    
    # valeures initiales des coeff
    u_n, u_nm1 = 0, 1 
    v_n, v_nm1 = 1, 0 

    while b!=0:
        q = a // b
        r = a % b
        # La relation de récurrence
        u_new = u_nm1 - q * u_n
        v_new = v_nm1 - q * v_n
        
        # mis à jour pour l'itération suivante
        u_n, u_nm1 = u_new, u_n
        v_n, v_nm1 = v_new, v_n
        a,b = b, r
        
    return a, u_nm1 , v_nm1

### Fonction pour l'étape 1.4)

In [4]:
def inv_mod(a,n):
    '''
    Calcul l'inverse de a modulo n en utilisant l'algorithme 
    d'Euclide étendu
    '''
    
    d, u, v = euclide_etendu(a, n)
    if d != 1:
        return False
    else:
        return u % n # pour avoir un représentant entre 0 et n-1

### Fonction pour l'étape 1.5)

In [5]:
def cle_rsa(c):
    ''' 
    Genere la cle privée (p;q;d) et la cle publique (e,n) en prenant 
    les deux premiers nombres premiers tels que p>c et q > 2c
    '''
    
    p = premier_sup(c)
    q = premier_sup(2*p)
    
    n = p * q
    phi = (p-1)*(q-1)
    
    e = n//4
    if e % 2==0: # si n/4 est pair
        e = e + 1
    
    e=3
    
    while True :
        d, u, v = euclide_etendu(e, phi)
        if d == 1: # e et phi sont premiers entre eux
            break
        e=e+2
    
    d=inv_mod(e,phi)
    
    return p,q,d,e,n

### Test temps d execution du programme CLE_RSA

In [7]:
import time as tm 

for i in range(1,5):
    c=10**i
    t1=tm.perf_counter()
    p,q,d,e,n = cle_rsa(c)
    t2=tm.perf_counter()
    print("c=10^",i)
    print("Cle privée (d;p;q)=(",d,";",p,";",q,")")
    print("Cle publique (e;n)=(",e,";",n,")")
    print("Le nombre de chiffres en base 10 de n est :",len(str(n))) 
    print("Temps de calculs des clés :",t2-t1,"secondes")
    print(" ")
    print("-------------- ")
    print(" ")

c=10^ 1
Cle privée (d;p;q)=( 147 ; 11 ; 23 )
Cle publique (e;n)=( 3 ; 253 )
Le nombre de chiffres en base 10 de n est : 3
Temps de calculs des clés : 1.1219999578315765e-05 secondes
 
-------------- 
 
c=10^ 2
Cle privée (d;p;q)=( 19091 ; 101 ; 211 )
Cle publique (e;n)=( 11 ; 21311 )
Le nombre de chiffres en base 10 de n est : 5
Temps de calculs des clés : 1.4149998605716974e-05 secondes
 
-------------- 
 
c=10^ 3
Cle privée (d;p;q)=( 1225325 ; 1009 ; 2027 )
Cle publique (e;n)=( 5 ; 2045243 )
Le nombre de chiffres en base 10 de n est : 7
Temps de calculs des clés : 1.5673002053517848e-05 secondes
 
-------------- 
 
c=10^ 4
Cle privée (d;p;q)=( 133546747 ; 10007 ; 20021 )
Cle publique (e;n)=( 3 ; 200350147 )
Le nombre de chiffres en base 10 de n est : 9
Temps de calculs des clés : 1.5632998838555068e-05 secondes
 
-------------- 
 


## Étape 2 : conversion par bloc et chiffrement RSA

### Fonction pour l'étape 2.1)

In [24]:
def txt_bloc(texte,b):
    '''
    Conversion d'une chaine de caractere en une 
    liste de nombres entiers par blocs de b chiffres
    (en base 10) avec b>=1 et à l'aide du code ASCII
    Attention: Les caractères doivent avoir un code
             ASCII entre 0 et 99, donc il faut utiliser 
             seulement des majuscules.
    '''
    
    # le code ascii du texte
    texte_ascii = [ord(c) for c in texte]
    
    # on ajoute des zéros pour avoir 
    # une longeur qui est un multiple de b
    r = len(texte) % b
    if r >0:
        texte_ascii = texte_ascii + [0]*(b-r)
    
    nb_blocs = len(texte_ascii) // b
    L=[]
    for k in range(nb_blocs):
        nombre = texte_ascii[k*b]
        for j in range(1,b):
            nombre = 100 * nombre + texte_ascii[k*b+j]
        L = L + [nombre]
    
    
    return L

txt_bloc("ESSAI",2)

[6983, 8365, 7300]

### Fonction pour l'étape 2.2)

In [23]:
def exp_rapide(x,k,n):
    """"Algorithme de l'exponentiation rapide
        pour le calcul de x^k mod n """
    c=1
    while k>0:
        if k%2==1:
            c=(c*x)%n
        x=(x*x)%n
        k=k//2
    return c


### Fonction pour l'étape 2.3)

In [28]:
def txt_rsa(texte,b,e,n):
    """"Conversion d'une chaine de caractère en 
    equivalent numérique ASCII par blocs de 
    longueurs b>=1 et ensuite chiffrement RSA 
    d'exposant e modulo n
    params:
        texte (str) : la chaîne de caractère à chiffrer
        b     (int) : la longueur des blocs de 2 chiffres
        (e, n)      : la clé privé
    return
        L : le message clair codé en ASII par blocs de 2*k chiffres
        M : le message chiffré
    """
    L = txt_bloc(texte,b)    
    M = [exp_rapide(x,e,n) for x in L]
    
    return L, M

## Étape 3 : Sécurité RSA et Cryptographie : Factorisation des entiers

### Fonction pour l'étape 3.2)

In [32]:
from math import sqrt

def ens_div_impair(n):
    """Ensemble de tous les diviseurs impairs
       d'un nombre entier n=p*q avec p et q 
       des nombres premiers """
    import math as mt
    
    L1=[1]
    L2=[n]
    for d in range(3,int(sqrt(n))+1,2):
        if n%d == 0:
            L1 = L1 + [d]
            L2 = [n//d] + L2
            
    if L1[-1] == L2[0]:
        return L1+L2[1:]
    else:
        return L1+L2

## Étape 4 : Déchiffrement RSA et conversion par Bloc

### Fonction pour l'étape 4.2)

In [33]:
def bloc_txt(L):
    """"Conversion d'une liste de nombres entiers 
    constitué de blocs de longueurs b>=1 en une 
    chaine de caratere à l'aide du code ASCII"""
    texte =""
    for nb in L:
        ch=str(nb)
        texte=texte+ch

    texte_final=""
    for i in range(len(texte)//2) :
        lettre=chr(eval(texte[2*i:2*(i+1)]))
        texte_final=texte_final+lettre

    for e in texte_final:
        if e=="*":
            texte_final=texte_final[:-1]
    return texte_final

### Fonction pour l'étape 4.3)

In [35]:
def rsa_text(L,d,n):
    """"Déchiffrement RSA d'exposant d modulo n
    et ensuite conversion des blocs de longueurs b>=1
    en chaine de caractère à l'aide du code ASCII """
    M=[]
    for elem in L:
        nbr=exp_rapide(elem,d,n)
        M.append(nbr)
    texte=bloc_txt(M)   
    return M,texte